In [1]:
import google.generativeai as genai
import json
import re
from tqdm import tqdm # 用於顯示進度條
import csv
import os

# --- 1. 常數與提示詞定義 (來自您的 GRPO 腳本) ---

SYSTEM_PROMPT = """你是一名專門根據患者用藥與病史紀錄預測 ICD-10 編碼的醫學專家。  
請嚴格遵守以下作答格式：  

<reasoning>  
逐步推理：逐條說明病例中每個臨床資訊（症狀、診斷、檢查結果、既往病史、用藥等）可能對應的 ICD-10 編碼，並在最後總結出最合理的編碼集合。  
</reasoning>  
<answer>  
僅輸出最終的 ICD-10 編碼，使用英文逗號分隔，不要添加任何額外文字或解釋。  
</answer>  
"""
EXAMPLE_QUESTION = """
Sex: F
Service: MEDICINE
Allergies: Aspirin
Attending: ___
Chief Complaint: weakness, diarrhea
Major Surgical or Invasive Procedure: None

History of Present Illness:
Ms. ___ is a ___ year-old woman with PMH significant for chronic anemia, osteoporosis, hypertension, ataxia, and recent L5 fracture in the setting of recurrent falls who presents from home with fatigue and generalized weakness and diarrhea.  

The patient's recent history is notable for the follow:  
- On ___, she presented with 4 days of LBP s/p fall from standing at which time imaging revealed acute L5 fracture. She was evaluated by Spine team who recommended early mobilization, pain control, but no brace required. She was evaluated by ___, and discharged to ___.  
- She was discharged home with ___ on ___.  
- On ___, she again presented to ___ s/p fall from standing while trying to reach for a glass of water. She did have a occipital scalp hematoma, but imaging including ___, C-spine CT, and L hip X-ray were negative for acute process so patient was discharged home.  

She now represents with generalized fatigue and diarrhea. In the setting of opiates for her L5 fracture, the patient has had constipation (5 days with no BM) for which she took a "natural laxative" the evening prior to presentation. The patient had 2 bowel movements in the morning of presentation and one episode of incontinence with diarrhea while sleeping. In this setting, she felt very weak and called EMS and was brought to ___ ED.

What ICD-10 codes should be assigned? Please just tell me which codes in the end
"""
EXAMPLE_REASONING = """The patient has chronic anemia, documented and clinically relevant, which corresponds to D500.  
There is vitamin deficiency suggested, which fits E538.  
She also has visual impairment history, which maps to H548.  
Hypertension is a chronic condition, coded as I10.  
Diarrhea due to laxative use is coded as K521 (toxic gastroenteritis and colitis).  
Her recent fracture with underlying osteoporosis corresponds to M810.  
Ataxia is a chronic neurologic condition, coded as R270.  
There is adverse effect of drugs (laxatives/opioids), so T474X5A applies.  
Environmental factor of injury is Y92099 (unspecified place of occurrence).  
She also has aspirin allergy, captured by Z9181.  

Therefore, the most appropriate ICD-10 codes are:
"""

EXAMPLE_ANSWER = "D500 E538 H548 I10 K521 M810 R270 T474X5A Y92099 Z9181"

c:\Users\Yuan\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def extract_icd_answer(text: str) -> str:
    """
    從模型的完整回覆或標準答案中提取 ICD-10 編碼。
    優先從 <answer> 標籤中提取,若無則從整個文本中提取。
    """
    # 使用正則表達式搜尋 <answer> 標籤，re.DOTALL 讓 . 可匹配換行符，re.IGNORECASE 忽略大小寫
    answer_match = re.search(r'<answer>(.*?)</answer>', text, re.DOTALL | re.IGNORECASE)
    # 如果找到 <answer> 標籤，則只在標籤內容中搜尋 ICD 編碼；否則在整個文本中搜尋
    search_text = answer_match.group(1) if answer_match else text
    
    # 匹配格式：字母 + 1-3位數字 + 可選的小數點和1-2位數字
    codes_list = re.findall(r'\b([A-Z]\d{1,6}(?:\.\d{1,2})?)\b', search_text, re.IGNORECASE)
    
    # 移除小數點、去重、排序、轉大寫，並過濾掉長度<=2的編碼
    cleaned_codes = sorted(set(
        code.replace('.', '').upper() 
        for code in codes_list 
        if len(code.replace('.', '')) > 2
    ))
    return " ".join(cleaned_codes)



def extract_xml_answer(text: str) -> str:
    """提取 <answer> 標籤內的內容"""
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    answer = answer.strip()
    return answer


def is_text(text: str) -> bool:
    """判斷輸入字串是否主要為文字敘述（而非 ICD code 等短編碼）"""
    letters_and_spaces = sum(c.isalpha() or c.isspace() for c in text)
    ratio = letters_and_spaces / max(1, len(text))  
    return ratio > 0.5


def compute_f1(pred: str, true: str) -> float:
    """
    計算 F1 分數 (前三個字符匹配即算正確)
    
    Args:
        pred: 預測的 ICD 編碼字串,例如 "A010 B021"
        true: 真實的 ICD 編碼字串,例如 "A010 C032"
    
    Returns:
        F1 分數 (0.0 到 1.0)
    """
    # 將字串轉換為 set
    pred = pred.replace('.', '')
    true = true.replace('.', '')
    
    def parse_codes(text):
        if ',' in text:
            return set(code.strip().upper() for code in text.split(',') if code.strip())
        else:
            return set(code.strip().upper() for code in text.split() if code.strip())
    
    pred_set = parse_codes(pred)
    true_set = parse_codes(true)
    
    # 如果都是空集合,視為完全匹配
    if not pred_set and not true_set:
        return 1.0
    
    # 轉換為前三個字符的集合
    def get_prefix_set(codes):
        return set(code[:3] for code in codes if len(code) >= 3)
    
    pred_prefix = get_prefix_set(pred_set)
    true_prefix = get_prefix_set(true_set)
    
    # 計算 TP, FP, FN
    tp = len(pred_prefix & true_prefix)
    fp = len(pred_prefix - true_prefix)
    fn = len(true_prefix - pred_prefix)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    
    if precision + recall == 0:
        return 0.0
    
    f1 = 2 * precision * recall / (precision + recall)
    return f1



def count_xml(text: str) -> float:
    """
    檢查 XML 標籤格式的完整性
    
    Returns:
        分數 (0.0 到 0.5),分數越高表示格式越完整
    """
    score = 0.0
    
    if text.count("<reasoning>\n") == 1:
        score += 0.125
    if text.count("\n</reasoning>\n") == 1:
        score += 0.125
    if text.count("\n<answer>\n") == 1:
        score += 0.125
        # 懲罰 </answer> 後有多餘內容
        extra_content = len(text.split("\n</answer>\n")[-1])
        score -= extra_content * 0.001
    if text.count("\n</answer>") == 1:
        score += 0.125
        # 懲罰 </answer> 後有多餘內容
        extra_content = len(text.split("\n</answer>")[-1]) - 1
        score -= max(0, extra_content) * 0.001
    
    return max(0.0, score)  # ✅ 確保不會是負數

In [ ]:
START_INDEX = 340     # 起始索引(從0開始)
END_INDEX = 505    # 結束索引(不包含此索引)

# --- 4. 載入 Gemini 模型 ---

genai.configure(api_key=" ")

print(f"✅ 正在設定模型: gemini-2.5-flash ...")
print(f"📊 處理區間: 第 {START_INDEX} 至 第 {END_INDEX-1} 筆 (共 {END_INDEX-START_INDEX} 筆)\n")

# generation_config = {
#     "temperature": 0,
#     "top_p": 1,
#     "top_k": 1,
#     "max_output_tokens": 8192,
# }

# 建立模型實例
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    system_instruction=SYSTEM_PROMPT
)

print("✅ 模型設定完畢。")

# --- 5. 載入並處理資料 ---
print("\n📂 正在載入 note_icd_data.jsonl ...")
chat_style_data = []

try:
    with open("note_icd_data.jsonl", "r", encoding="utf-8") as f:
        all_lines = f.readlines()
        total_records = len(all_lines)
        
        print(f"📊 檔案總共有 {total_records} 筆資料")
        
        # 驗證索引範圍
        if START_INDEX < 0 or END_INDEX > total_records:
            print(f"❌ 錯誤：索引範圍超出資料範圍 (0-{total_records})")
            exit()
        
        if START_INDEX >= END_INDEX:
            print(f"❌ 錯誤：起始索引必須小於結束索引")
            exit()
        
        # 只讀取指定區間的資料
        lines = all_lines[START_INDEX:END_INDEX]
        print(f"✅ 已選取第 {START_INDEX} 至 第 {END_INDEX-1} 筆，共 {len(lines)} 筆\n")

        for line in tqdm(lines, desc="[Gemini] 載入並預處理資料"):
            item = json.loads(line)
            ground_truth_answer = extract_icd_answer(item["answer"])
            
            chat_style_data.append({
                "answer": ground_truth_answer,
                "raw_question": item["question"]
            })

except FileNotFoundError:
    print("❌ 錯誤：找不到 'note_icd_data.jsonl' 檔案。")
    exit()

print(f"✅ 資料載入完畢，共 {len(chat_style_data)} 筆。\n")

✅ 正在設定模型: gemini-2.5-flash ...
📊 處理區間: 第 340 至 第 504 筆 (共 165 筆)

✅ 模型設定完畢。

📂 正在載入 note_icd_data.jsonl ...
📊 檔案總共有 3852 筆資料
✅ 已選取第 340 至 第 504 筆，共 165 筆



[Gemini] 載入並預處理資料: 100%|██████████| 165/165 [00:00<00:00, 39709.67it/s]

✅ 資料載入完畢，共 165 筆。



In [4]:
# --- 6. 執行推論並計算指標 ---
all_predictions = []
all_ground_truths = []
all_f1_scores = []
all_completion_lengths = []

# 生成帶區間標記的 CSV 檔名
csv_filename = f"gemini_flash_results_{START_INDEX}_{END_INDEX}.csv"

# 檢查檔案是否已存在
file_exists = os.path.exists(csv_filename)

with open(csv_filename, 'a', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['index', 'accuracy_reward', 'completion_length', 'ground_truths_ans', 'predictions_ans', 'assistant_reply']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    # 只在檔案不存在時寫入表頭
    if not file_exists:
        writer.writeheader()
        print(f"📁 CSV 檔案已建立: {csv_filename}")
    else:
        print(f"📁 CSV 檔案已存在，將追加資料: {csv_filename}")
    
    print(f"📊 開始處理區間 {START_INDEX}-{END_INDEX-1}\n")
    
    # 開始推論
    for idx, item in enumerate(tqdm(chat_style_data, desc="[Gemini] 執行推論與評估"), START_INDEX):
        try:
            question = item["raw_question"]
            gt_answer = item["answer"]
            
            # 建立包含 one-shot 範例的完整對話
            chat = model.start_chat(history=[
                {"role": "user", "parts": [EXAMPLE_QUESTION]},
                {"role": "model", "parts": [f"<reasoning>{EXAMPLE_REASONING}</reasoning><answer>{EXAMPLE_ANSWER}</answer>"]}
            ])
            
            # 發送問題並取得回應
            response = chat.send_message(question)
            assistant_reply = response.text
            
            # 印出完整資訊
            print("\n" + "="*80)
            print(f"📝 樣本 #{idx} (絕對索引)")
            print("="*80)
            print(f"\n🤖 模型完整回答:")
            print("-"*80)
            print(assistant_reply)
            print("-"*80)
            
            # 記錄 ground truth
            all_ground_truths.append(gt_answer)
            
            # 計算 completion length
            completion_length = len(assistant_reply)
            print(f"\n📊 完成的回覆長度 (字元數): {completion_length}")
            all_completion_lengths.append(completion_length)
            
            # 提取預測的 ICD 編碼
            predicted_codes = extract_icd_answer(assistant_reply)
            print(f"🔍 提取的 ICD-10 編碼: {predicted_codes}")
            all_predictions.append(predicted_codes)
            
            print(f"\n🎯 Ground Truth Answer:")
            print(f"   {gt_answer}")
            
            # 計算 F1 分數
            f1_score = compute_f1(predicted_codes, gt_answer)
            print(f"✅ F1 分數: {f1_score:.4f}")
            all_f1_scores.append(f1_score)
            
            # 寫入 CSV (包含絕對索引)
            writer.writerow({
                'index': idx,
                'accuracy_reward': f1_score,    
                'completion_length': completion_length,
                'ground_truths_ans': gt_answer if isinstance(gt_answer, str) else json.dumps(gt_answer), 
                'predictions_ans': json.dumps(predicted_codes) if isinstance(predicted_codes, list) else predicted_codes,
                'assistant_reply': assistant_reply if isinstance(assistant_reply, str) else json.dumps(assistant_reply)
            })
            csv_file.flush()
            
            print(f"💾 結果已儲存至 {csv_filename} (第 {idx} 筆)")
            print("="*80 + "\n")
            
        except Exception as e:
            print(f"\n❌ 處理第 {idx} 筆資料時發生錯誤:")
            print(f"   錯誤訊息: {e}")
            print(f"   跳過此筆資料繼續執行...\n")
            continue

print("\n" + "="*80)
print("🎉 推論完成!")
print("="*80)

# --- 7. 顯示最終統計 ---
if not all_predictions:
    print("⚠️  沒有可評估的結果。")
else:
    print("\n" + "="*50)
    print(f"  Gemini-2.5-Flash 區間統計 [{START_INDEX}-{END_INDEX-1}]")
    print("="*50)
    print(f"✅ 總共處理: {len(all_f1_scores)} 筆資料")
    print(f"📊 平均 F1 分數: {sum(all_f1_scores)/len(all_f1_scores):.4f}")
    print(f"📏 平均回覆長度: {sum(all_completion_lengths)/len(all_completion_lengths):.1f} 字元")
    print(f"💾 結果已完整儲存至: {csv_filename}")
    print("="*50)

📁 CSV 檔案已建立: gemini_flash_results_340_505.csv
📊 開始處理區間 340-504



[Gemini] 執行推論與評估:   1%|          | 1/165 [00:20<56:50, 20.80s/it]


📝 樣本 #340 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>
逐步推理：
1.  **Chief Complaint: Toe ulcer**：患者有非癒合的慢性右腳趾傷口，這是糖尿病的常見併發症。
2.  **Past Medical History: Type II DM**：這是主要病史，且有多項併發症。
    *   **Diabetic foot ulcer**：E11.621 (Type 2 diabetes mellitus with diabetic foot ulcer) 最能反映糖尿病足潰瘍。
    *   **Diabetic peripheral angiopathy**：患者有間歇性跛行，且進行了血管繞道手術，這表明存在糖尿病性周邊動脈病變。E11.51 (Type 2 diabetes mellitus with diabetic peripheral angiopathy without gangrene)。
3.  **Past Medical History: Hypertension**：高血壓是既往病史，需要編碼。
    *   I10 (Essential (primary) hypertension)。
4.  **Past Medical History: Stroke**：腦中風是既往病史，未提及急性發作或特定後遺症影響本次就診。
    *   Z86.73 (Personal history of cerebral infarction without residual deficits)。
5.  **Past Medical History: Tobacco abuse**：煙草濫用是一個重要的風險因素。
    *   F17.210 (Nicotine dependence, cigarettes, uncomplicated) 通常用於煙草濫用。
6.  **History of Present Illness: Non-healing chronic right toe wound with purulent disch

[Gemini] 執行推論與評估:   1%|          | 2/165 [00:42<57:09, 21.04s/it]


📝 樣本 #341 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>
逐步推理：
1.  **AMS (Altered Mental Status) / Benzodiazepine Overdose / Delirium:** 患者主訴 AMS，並自述因失眠服用過量 Klonopin (Clonazepam)，導致意識不清及跌倒。精神科評估為苯二氮平類藥物中毒，並有消退性譫妄。由於患者有鎮靜催眠藥物依賴史，此情況應編碼為藥物中毒及其引起的譫妄。
    *   **T42.4X1A**：苯二氮平類藥物中毒，意外（非故意），初次就診 (Poisoning by benzodiazepines, accidental (unintentional), initial encounter)。
    *   **F13.22**：鎮靜劑、催眠劑或抗焦慮劑依賴伴有中毒性譫妄 (Sedative, hypnotic, or anxiolytic dependence with intoxication delirium)。這個碼同時涵蓋了依賴和急性中毒性譫妄，解釋了 AMS。
2.  **Fall:** 患者在半夜起床上廁所時跌倒。儘管影像學檢查未發現急性損傷，但跌倒本身仍需編碼。
    *   **W01.XXXA**：因滑倒、絆倒或跌倒而在同一水平面上跌倒，初次就診 (Fall on same level from slipping, tripping and stumbling, initial encounter)。
3.  **Insomnia:** 患者表示因使用類固醇導致睡眠困難，是服用過量 Klonopin 的直接原因。
    *   **G47.00**：失眠，未特指 (Insomnia, unspecified)。
4.  **Adverse effect of Prednisone:** 類固醇 (Prednisone) 引起失眠，是本次事件的促成因素之一。
    *   **T38.0X5A**：糖皮質激素和合成類似物的不良反應，初次就診 (Adverse effect o

...


📝 樣本 #502 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>
逐步推理：
1.  **abdominal pain and back pain**: 這些是主訴症狀，但一旦發現潛在的診斷（胸主動脈剝離），通常不會單獨編碼這些症狀，因為原因已經明確。
2.  **h/o HTN (Hypertension)**: 患者有高血壓病史，且入院時血壓高達174/103，這是一個需要編碼的慢性疾病，也可能是導致主動脈剝離的風險因素。對應 ICD-10 編碼為 **I10** (Essential (primary) hypertension)。
3.  **bradycardia s/p pacemaker**: 患者有心跳過緩病史並植入心律調節器。雖然病歷中提到心跳過緩，但由於植入心律調節器，通常會編碼心律調節器的存在狀態而非心跳過緩本身，除非心跳過緩在調節器存在下仍是活動性問題。對應 ICD-10 編碼為 **Z95.4** (Presence of cardiac pacemaker)。
4.  **known infrarenal AAA (Abdominal Aortic Aneurysm)**: 患者有已知的腎下腹主動脈瘤。病歷中沒有指出其破裂或與當前症狀直接相關，因此編碼為未破裂。對應 ICD-10 編碼為 **I71.4** (Aortic aneurysm, infrarenal part, without rupture)。
5.  **new descending thoracic dissection**: 這是患者此次就診的主要急性診斷，由CT確認。這是最嚴重的、導致主訴的原因。對應 ICD-10 編碼為 **I71.2** (Aortic dissection, thoracic, unspecified part)。儘管提及"descending"，但ICD-10中沒有更精確的「降主動脈剝離」分碼，因此「胸主動脈剝離，未明確部位」是最適當的。
6.  **SBP upon arrival was 174/103**: 這表示入院時有高血壓，支持編碼高血壓 (I10)。

[Gemini] 執行推論與評估:  99%|█████████▉| 164/165 [44:05<00:19, 19.43s/it]


📝 樣本 #503 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>
逐步推理：
1.  **generalized weakness/falls (Chief Complaint):** 患者以全身無力及跌倒為主訴。
    *   `R53.1` (Weakness) 用於描述全身無力。
    *   `R29.7X1` (Falls, recurrent) 用於描述復發性跌倒，患者在過去一年中有6次跌倒。
2.  **CAD s/p multivessel CABG:** 冠狀動脈疾病並接受多血管冠狀動脈繞道手術史。
    *   `I25.10` (Atherosclerotic heart disease of native coronary artery without angina pectoris) 用於冠狀動脈疾病。
    *   `Z95.1` (Presence of aortocoronary bypass graft) 用於描述冠狀動脈繞道手術後的狀態。
3.  **a-fib on coumadin:** 心房顫動並服用Coumadin。
    *   `I48.91` (Unspecified atrial fibrillation) 用於心房顫動。
    *   `Z79.01` (Long-term (current) use of anticoagulants) 用於長期服用抗凝劑。
4.  **AICD placement after v-fib arrest:** 植入式心臟去顫器放置史，因心室顫動停止。
    *   `Z95.810` (Presence of automatic implantable cardiac defibrillator) 用於描述自動植入式心臟去顫器的存在。
5.  **spinal stenosis:** 脊椎狹窄病史。
    *   `M48.00` (Spinal stenosis, site unspecified) 因未指明具體部位。
6.  **injured his knee exercising

[Gemini] 執行推論與評估: 100%|██████████| 165/165 [44:13<00:00, 16.08s/it]


📝 樣本 #504 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>
逐步推理：
1. **Chief Complaint and History of Present Illness**: "6.9 cm multi-cystic lesion in the interpolar region of the left kidney". This describes a mass or lesion in the kidney.
2. **Major Surgical or Invasive Procedure**: "Robot-assisted laparoscopic left partial nephrectomy". The performance of a partial nephrectomy for a 6.9 cm multi-cystic lesion strongly suggests that the nature of the lesion was uncertain or suspicious enough to warrant surgical removal for diagnosis and treatment.
3. **ICD-10 Code Selection**:
    * Simple renal cysts (N28.1) are typically managed conservatively unless symptomatic or very large, and "multi-cystic" implies complexity beyond a simple cyst.
    * Benign neoplasms (D30.0) would be assigned if a definitive benign pathology was confirmed *before* or *after* the procedure, but the record doesn't state this.
    * Malignant neopl

In [5]:
import re

def extract_icd_answer(text: str) -> str:
    """
    從模型的完整回覆或標準答案中提取 ICD-10 編碼。
    優先從 <answer> 標籤中提取,若無則從整個文本中提取。
    """
    # 使用正則表達式搜尋 <answer> 標籤，re.DOTALL 讓 . 可匹配換行符，re.IGNORECASE 忽略大小寫
    answer_match = re.search(r'<answer>(.*?)</answer>', text, re.DOTALL | re.IGNORECASE)
    # 如果找到 <answer> 標籤，則只在標籤內容中搜尋 ICD 編碼；否則在整個文本中搜尋
    search_text = answer_match.group(1) if answer_match else text
    
    # 匹配格式：字母 + 1-3位數字 + 可選的小數點和1-2位數字
    codes_list = re.findall(r'\b([A-Z]\d{1,6}(?:\.\d{1,2})?)\b', search_text, re.IGNORECASE)
    
    # 移除小數點、去重、排序、轉大寫，並過濾掉長度<=2的編碼
    cleaned_codes = sorted(set(
        code.replace('.', '').upper() 
        for code in codes_list 
        if len(code.replace('.', '')) > 2
    ))
    return " ".join(cleaned_codes)


# 測試字串
test_text = """D550,O09212,O26872,O30042,O99012,Z3A18。"""

# 執行測試
result = extract_icd_answer(test_text)
print(f"提取結果: {result}")
print(f"預期結果: D550 O09212 O26872 O30042 O99012 Z3A18")
print(f"測試通過: {result == 'R074 R109'}")

提取結果: D550 O09212 O26872 O30042 O99012
預期結果: D550 O09212 O26872 O30042 O99012 Z3A18
測試通過: False
